In [ ]:
!pip install -qU "semantic-router[qdrant]"

In [5]:
from semantic_router import Route

# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

In [6]:
import os
from getpass import getpass
from semantic_router.encoders import CohereEncoder

os.environ["COHERE_API_KEY"] = os.environ.get("COHERE_API_KEY") or getpass(
    "Enter COHERE API key: "
)
encoder = CohereEncoder()

In [7]:
from semantic_router.index.qdrant import QdrantIndex


qd_index = QdrantIndex(location=":memory:")

In [8]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=qd_index)

2024-03-27 18:22:42 INFO semantic_router.utils.logger local


We can check our route layer and index information.

In [9]:
rl.list_route_names()

['politics', 'chitchat']

In [10]:
len(rl.index)

10

And query:

In [12]:
rl("don't you love politics?").name

'politics'

In [13]:
rl("how's the weather today?").name

'chitchat'

In [ ]:
rl("I'm interested in learning about llama 2").name

We can delete or update routes.

In [14]:
len(rl.index)

10

In [15]:
import time

rl.delete(route_name="chitchat")
time.sleep(1)
len(rl.index)

5

In [16]:
rl("how's the weather today?").name

In [17]:
rl.index.get_routes()

[('politics', 'they will save the country!'),
 ('politics', "isn't politics the best thing ever"),
 ('politics', "why don't you tell me about your political opinions"),
 ('politics', "they're going to destroy this country!"),
 ('politics',
  "don't you just love the presidentdon't you just hate the president")]

In [18]:
rl.index.describe()

{'type': 'qdrant', 'dimensions': 1024, 'vectors': 5}